#INIT

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

#BUSINESS TRANSFORMATION AND MODELING

In [0]:
%sql
DROP VIEW if EXISTS gold.dim_customers

In [0]:
%sql
SELECT 
  ROW_NUMBER() OVER (ORDER BY ci.Customer_id) AS Customer_number,
  ci.Customer_id,
  ci.Customer_key,
  ci.Firstname,
  ci.Lastname,
  ci.Marital_status,
  CASE WHEN ci.Gender != 'n/a' THEN ci.Gender -- CRM is the Master Gender Info
      ELSE COALESCE(cu.Gender, 'n/a')
  END AS Gender,
  lo.Country,
  cu.BirthDate,
  ci.Create_date
FROM silver.crm_cust_info ci
LEFT JOIN silver.erp_cust_az12 cu 
ON ci.Customer_key = cu.CustomerID
LEFT JOIN silver.erp_loc_a101 lo 
ON ci.Customer_key = lo.CustomerID

#WRITE IT TO GOLD TABLE

In [0]:
%sql
CREATE VIEW IF NOT EXISTS gold.dim_customers AS 
SELECT 
  ROW_NUMBER() OVER (ORDER BY ci.Customer_id) AS Customer_number,
  ci.Customer_id,
  ci.Customer_key,
  ci.Firstname,
  ci.Lastname,
  ci.Marital_status,
  CASE WHEN ci.Gender != 'n/a' THEN ci.Gender -- CRM is the Master Gender Info
      ELSE COALESCE(cu.Gender, 'n/a')
  END AS Gender,
  lo.Country,
  cu.BirthDate,
  ci.Create_date
FROM silver.crm_cust_info ci
LEFT JOIN silver.erp_cust_az12 cu 
ON ci.Customer_key = cu.CustomerID
LEFT JOIN silver.erp_loc_a101 lo 
ON ci.Customer_key = lo.CustomerID

#TESTING

In [0]:
%sql
SELECT * FROM gold.dim_customers